In [35]:
import pandas as pd
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import numpy as np

In [49]:
train = pd.read_csv("train_data.csv")
test = pd.read_csv("test_data.csv")
staions = pd.read_csv("station_list.csv")

In [37]:
train.head()

,id,name,host_id,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,last_review,reviews_per_month,availability_365,y
0,1,KiyosumiShirakawa 3min|★SkyTree★|WIFI|Max4|Tre...,242899459,Koto Ku,35.68185,139.80310,Entire home/apt,1,55,2020-04-25,2.21,173,12008
1,2,Downtown Tokyo Iriya next to Ueno,308879948,Taito Ku,35.72063,139.78536,Entire home/apt,6,72,2020-03-25,2.11,9,6667
2,3,"Japan Style,Private,Affordable,4min to Sta.",300877823,Katsushika Ku,35.74723,139.82349,Entire home/apt,1,18,2020-03-23,3.46,288,9923
3,4,4 min to Shinjuku Sta. by train / 2 ppl / Wi-fi,236935461,Shibuya Ku,35.68456,139.68077,Entire home/apt,1,2,2020-04-02,1.76,87,8109
4,5,LICENSED SHINJUKU HOUSE: Heart of the action!,243408889,Shinjuku Ku,35.69840,139.70467,Entire home/apt,1,86,2020-01-30,2.00,156,100390


In [38]:
train["reviews_per_month"] = train["reviews_per_month"].fillna(0)
train["last_review"] = train["last_review"].fillna("1900-01-01")
test["reviews_per_month"] = test["reviews_per_month"].fillna(0)

In [39]:
int_columns = train.select_dtypes(exclude="object").columns
X_train,y_train = train[int_columns].drop("y",axis=1),train["y"]
X_test = test.select_dtypes(exclude="object")

In [40]:
X_test.isnull().sum()

id                   0
host_id              0
latitude             0
longitude            0
minimum_nights       0
number_of_reviews    0
reviews_per_month    0
availability_365     0
dtype: int64

In [41]:
model = RFR()

In [42]:
#定義したX_train,y_trainを入れる
scores = []
kf = KFold(n_splits=5, shuffle=True, random_state=1)
for tr_idx, va_idx in kf.split(X_train):
    tr_x, va_x = X_train.iloc[tr_idx], X_train.iloc[va_idx]
    tr_y, va_y = y_train.iloc[tr_idx], y_train.iloc[va_idx]
    model.fit(tr_x, tr_y)
    va_pred = model.predict(va_x)
    rmsle = np.sqrt(mean_squared_error(np.log1p(va_y), np.log1p(va_pred)))
    scores.append(rmsle)
    print(rmsle)

# クロスバリデーションの平均のスコアを出力する
print(f'RMSLE: {np.mean(scores):.4f}')

0.8639817503018608
0.8606188427229623
0.8158771467113151
0.8441021379026923
0.8368882024916312
RMSLE: 0.8443


In [43]:
model.fit(X_train,y_train)

RandomForestRegressor()

In [44]:
X_train.head()

,id,host_id,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,availability_365
0,1,242899459,35.68185,139.80310,1,55,2.21,173
1,2,308879948,35.72063,139.78536,6,72,2.11,9
2,3,300877823,35.74723,139.82349,1,18,3.46,288
3,4,236935461,35.68456,139.68077,1,2,1.76,87
4,5,243408889,35.69840,139.70467,1,86,2.00,156


In [45]:
X_test

,id,host_id,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,availability_365
0,1,184730720,35.67131,139.73285,1,0,0.00,183
1,2,20993205,35.73014,139.71739,2,21,1.94,337
2,3,322521715,35.66193,139.66540,1,14,0.82,240
3,4,234477095,35.73603,139.74794,1,16,1.19,0
4,5,145453833,35.72126,139.78320,30,2,0.19,164
...,...,...,...,...,...,...,...,...
4991,4992,166712541,35.68555,139.68015,2,31,1.60,174
4992,4993,58261300,35.72499,139.77741,2,6,0.27,90
4993,4994,243229864,35.69102,139.70614,1,12,2.54,113
4994,4995,50190468,35.69848,139.70435,2,71,0.96,273


In [46]:
y_pred = model.predict(X_test)

In [53]:
submission = pd.DataFrame({"id":test["id"],"y":y_pred})
submission

,id,y
0,1,30725.97
1,2,32290.95
2,3,32430.93
3,4,34662.22
4,5,39762.80
...,...,...
4991,4992,23591.52
4992,4993,15812.17
4993,4994,14163.55
4994,4995,22686.20


In [54]:
submission.to_csv("submission.csv",index=False)